# 散户反买策略

In [3]:
# 数据接口 
import akshare as ak
import baostock as bs
import tushare as ts

# 基础模块
import datetime
from itertools import product
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import requests
import time

# 回测框架
import backtrader as bt

# 基础函数
import utilsJ

## 策略

### 数据槽

In [ ]:
class Data_MoneyFlow(bt.feeds.PandasData):
    '''
    The ``dataname`` parameter inherited from ``feed.DataBase`` is the pandas
    DataFrame
    '''
    lines = ('buy_sm_amount', 'sell_sm_amount', 'buy_md_amount', 'sell_md_amount', 
             'buy_lg_amount', 'sell_lg_amount', 'buy_elg_amount', 'sell_elg_amount',
    )

    params = (
        # Possible values for datetime (must always be present)
        #  None : datetime is the "index" in the Pandas Dataframe
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('datetime', None),

        # Possible values below:
        #  None : column not present
        #  -1 : autodetect position or case-wise equal name
        #  >= 0 : numeric index to the colum in the pandas dataframe
        #  string : column name (as index) in the pandas dataframe
        ('open', -1),
        ('high', -1),
        ('low', -1),
        ('close', -1),
        ('volume', -1),
        ('openinterest', -1),
        ('sm_net_amount', -1),
        ('sm_net_percent', -1),
        ('md_net_amount', -1),
        ('md_net_percent', -1),
        ('lg_net_amount', -1),
        ('lg_net_percent', -1),
        ('elg_net_amount', -1),
        ('elg_net_percent', -1),
        ('major_net_amount', -1),
        ('major_net_percent', -1)
    )

### 策略主体

In [ ]:
class opposite(bt.Strategy):
    
    params = (
        # General params
        ('printlog', False),
        ('units', 1),
        ('p_stake', 100),
    )
    
    
    def log(self, txt, dt=None, doprint=False):
        if self.params.printlog or doprint:
            dt = dt or self.datas[0].datetime.date(0)
            print('%s: %s' % (dt.isoformat(), txt))
            #with open('log.txt', 'a') as file:
                #file.write('%s: %s \n' % (dt.isoformat(), txt))
        
    
    def __init__(self):
        # Keep references to lines in the data[0] dataseries
        self.dataclose = self.datas[0].close
        self.datahigh = self.datas[0].high
        self.datalow = self.datas[0].low

        # Keep references to executed order & price
        self.buyprice = 0
        self.sellprice = 0
        self.order = None


    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            return

        if order.status in [order.Completed]:
            if order.isbuy():
                self.buyprice = order.executed.price
                self.log('BUY EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                         (order.executed.price,
                          order.executed.size,
                          self.broker.get_cash()))

            else:
                self.sellprice = order.executed.price
                self.log('SELL EXECUTED, Price:%.2f, Lot:%i, Cash:%i.' %
                        (order.executed.price,
                          -order.executed.size,
                          self.broker.get_cash()))

        elif order.status in [order.Canceled, order.Margin, order.Rejected]:
            self.log('Order Canceled/Margin/Rejected')

        # Write down: no pending order
        self.order = None


    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))


    def next(self):
        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return
        print(self.datas[0].datetime.date(0))
        if self.datas[0].datetime.date(0).month == 3:
            print(self.datas[0].ROE[0])
        else:
            print(self.datas[0].ROE[0] - self.datas[0].ROE[-1])


    #def stop(self):
    #    self.log('Ending Position:%i. Ending Value:%.2f.' %
    #            (self.getposition(self.data).size,
    #            self.cerebro.broker.getvalue()), doprint=False)

## 测试运行

In [4]:
def moneyflow_ts(token, stock_code, start, end):
    ts.set_token(token)
    df = ak.stock_individual_fund_flow(ts_code=stock_code)
    df.index=pd.to_datetime(df.trade_date)
    df.drop(['ts_code','trade_date'], axis=1, inplace = True)
    df = df.astype('float')
    return df[::-1]

In [ ]:
s_date = datetime.date(2020,12,31) - datetime.timedelta(days=365)
e_date = datetime.date(2020,12,31)
token = '74f1379591c9d810854fa5891fffcacaba514b82bf17ec2e239025b6'
stock_code = '002514.SZ'

if __name__ == '__main__':
    moneyflow_ts(token, stock_code, s_date, e_date)

# 爬虫测试

In [17]:
'002514.SZ'[:6]

'002514'

In [27]:
def money_flow(stock_code):
    sec_id = '0.' + stock_code[:6] if stock_code[0] != '6' else '1.' + stock_code[:6]
    headers = {'Accept': '*/*',
            'Accept-Language': 'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
            'Host': 'push2his.eastmoney.com',
            'Referer': 'https://data.eastmoney.com/',
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/106.0.0.0 Safari/537.36 Edg/106.0.1370.47',
                }

    f_dict = {'f51': 'trade_date',
            'f52': 'major_net_amount',
            'f53': 'sm_net_amount',
            'f54': 'md_net_amount',
            'f55': 'lg_net_amount',
            'f56': 'elg_net_amount',
            'f57': 'major_net_percent',
            'f58': 'sm_net_percent',
            'f59': 'md_net_percent',
            'f60': 'lg_net_percent',
            'f61': 'elg_net_percent'
        }

    params = (
            ('lmt', '0'),
            ('klt', '101'),
            ('secid','0.002514'),
            ('fields1', 'f1,f2,f3,f7'),
            ('fields2', ",".join(f_dict.keys())),
        )
    params = dict(params)
    url = 'http://push2his.eastmoney.com/api/qt/stock/fflow/daykline/get'
    json_response = requests.get(url,
                                    headers=headers, params=params).json()
    data = json_response.get('data')
    df = pd.DataFrame([x.split(',') for x in data['klines']], columns=f_dict.values())
    df['trade_date'] = df.trade_date.apply(lambda x: datetime.datetime.strptime(x, '%Y-%m-%d'))
    return  df

In [31]:
money_flow('002514.SZ')

,trade_date,major_net_amount,sm_net_amount,md_net_amount,lg_net_amount,elg_net_amount,major_net_percent,sm_net_percent,md_net_percent,lg_net_percent,elg_net_percent
0,2022-05-19,-10059669.0,10884615.0,-824947.0,-10059669.0,0.0,-27.05,29.27,-2.22,-27.05,0.00
1,2022-05-20,-4080854.0,3281618.0,799236.0,-2919574.0,-1161280.0,-18.49,14.87,3.62,-13.23,-5.26
2,2022-05-23,17224902.0,-9056832.0,-8168071.0,1307560.0,15917342.0,31.27,-16.44,-14.83,2.37,28.90
3,2022-05-24,17276524.0,-19070697.0,1794172.0,13562318.0,3714206.0,9.15,-10.10,0.95,7.18,1.97
4,2022-05-25,1033672.0,-4283578.0,3249907.0,1033672.0,0.0,0.93,-3.86,2.93,0.93,0.00
...,...,...,...,...,...,...,...,...,...,...,...
97,2022-10-12,66905040.0,8926096.0,-75831152.0,-134311776.0,201216816.0,3.17,0.42,-3.59,-6.36,9.53
98,2022-10-13,-413511885.0,356557936.0,56953968.0,-195636816.0,-217875069.0,-19.02,16.40,2.62,-9.00,-10.02
99,2022-10-14,-77575012.0,39079920.0,38495120.0,-1892112.0,-75682900.0,-4.08,2.05,2.02,-0.10,-3.98
100,2022-10-17,70560377.0,10448688.0,-81009056.0,-71345344.0,141905721.0,3.58,0.53,-4.11,-3.62,7.20
